In [ ]:
import numpy as np
import boto3
import tensorflow as tf
import pandas as pd
import os
import csv
import cStringIO
! sudo pip install s3fs
import s3fs
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [66]:
def sort_pd_df_by_ext_vec(df, ext_sor_vec, cols):
    df_s = df[((df[cols[0]]).astype(int)).isin(ext_sor_vec)] #
    df_s['sort_cat'] = pd.Categorical(df_s[cols[0]],categories = ext_sor_vec,ordered = True)
    if len(cols) > 1:
        df_s.sort_values(['sort_cat',cols[1]] ,inplace = True)
    
    else:
        df_s.sort_values(['sort_cat'],inplace = True) 
    
    df_s.reset_index(inplace = True)
    df_ = df_s.drop(['sort_cat','index'] ,axis = 1)
    
    return(df_)

In [58]:
def trainSamples(viewers,videos,probab,viewerFeat,videoFeat,contxFeat):
    trData = {} # trData = Dictionary with training data. This is histrory of viewer and video iteraction
    X = {} # X  = Dictionary with viewer features as arrays
    Y ={} # Y  = Dictionary with video features as arrays
    for i in range(viewers):
        X[i] = np.random.rand(viewerFeat)
        a = 0 # timing of the video for a particular user,... 
                #to give the order in which the videos have been watched
        for j in range(videos):
            if int(np.random.binomial(1,probab ,1)[0]):
                trData[(i,j,a)] = np.random.rand(contxFeat)
                a+=1 # when a video is watched, we increase the value of a by 1 
            if i==0:
                Y[j] = np.random.rand(videoFeat)
    return X,Y,trData

In [59]:
viewers = 1000  #number of viewers
videos =1000  #number of videos
probab = 0.05  #probability of a viewer watching any one video
viewerFeat = 310  #number of features describing a veiwer
videoFeat = 300   #number of features describing a video
contxFeat = 15 # number of contextual features
# X  = Dictionary with viewer features as arrays
# Y  = Dictionary with video features as arrays
# trData = Dictionary with training data. This is histrory of viewer and video iteraction
X ,Y ,trData = trainSamples(viewers ,videos ,probab ,viewerFeat ,videoFeat ,contxFeat)

In [60]:
user_feat_inp = np.array([X[key] for key in sorted(X.keys())]) 
vid_feat_inp = np.array([Y[key] for key in sorted(Y.keys())])

In [61]:
key_user = np.asarray(range(user_feat_inp.shape[0])).reshape(user_feat_inp.shape[0] ,1)
key_vid = np.asarray(range(vid_feat_inp.shape[0])).reshape(vid_feat_inp.shape[0] ,1)
user_feat_inp_w_key = np.concatenate((user_feat_inp ,key_user),axis = 1)
vid_feat_inp_w_key = np.concatenate((vid_feat_inp ,key_vid),axis = 1)
user_vid_time = trData.keys()

In [62]:
user_feat_inp_w_key_df = pd.DataFrame(user_feat_inp_w_key) 
vid_feat_inp_w_key_df = pd.DataFrame(vid_feat_inp_w_key)
user_vid_time_df = pd.DataFrame(user_vid_time)

In [63]:
rr = user_vid_time_df.sort_values([0 ,2]).reset_index()

In [64]:
h = pd.get_dummies(rr[1],prefix = 'vid_')
y_tr_p_w = pd.concat([rr.reset_index(drop = True), h], axis = 1)
y_tr_p_w.rename(columns = {0: 'user'}, inplace = True)
y_tr_p_w.rename(columns = {1: 'mov'}, inplace = True)
y_tr_p_w.rename(columns = {2: 'rank'}, inplace = True)
y_tr_p_w['desired'] = np.argmax(np.array(y_tr_p_w.iloc[:,4:]) ,1)

In [65]:
max_watch = (y_tr_p_w.groupby('user',axis = 0).sum().iloc[:
                                ,int(np.array(np.where(y_tr_p_w.columns=='rank'))):-1].sum(axis=1)).reset_index()

In [66]:
vid_feat_inp_w_key_df.rename(columns = {300: 'mov_id'}, inplace = True)
user_feat_inp_w_key_df.rename(columns = {310: 'user_id'}, inplace = True)
user_vid_time_df.rename(columns = {0: 'user_id'}, inplace = True)
user_vid_time_df.rename(columns = {1: 'mov_id'}, inplace = True)
user_w_vid_tim_and_feat = user_vid_time_df.merge(user_feat_inp_w_key_df 
                                                 ,how = 'inner',on = 'user_id', sort = False)
user_vid_time_vidfeat_usefit = user_w_vid_tim_and_feat.merge(vid_feat_inp_w_key_df 
                                                             ,how = 'inner' ,on='mov_id' ,sort = False)
user_vid_time_vidfeat_usefit.rename(columns={'2_x': 'time_watch'} ,inplace=True)
user_vid_time_vidfeat_usefit_sorted = user_vid_time_vidfeat_usefit.sort_values(['user_id', 'time_watch'])

In [67]:
tr_users = viewers * 0.8
tr_y = y_tr_p_w[y_tr_p_w['user'] < tr_users]
te_y = y_tr_p_w[~y_tr_p_w['user'].isin(tr_y['user'])]

In [46]:
b = brand_movie_genre.sort_values("conEpIndex")

In [101]:
brand_movie_genre[brand_movie_genre["brand"]=='come-dine-with-me']
#made-in-chelsea
#a-place-in-the-sun


,conEpID,conEpIndex,brand,genre
4,47822_100,2495,come-dine-with-me,Lifestyle
6,50823_10,6533,come-dine-with-me,Lifestyle
8,51382_115,4591,come-dine-with-me,Lifestyle
9,51382_133,338,come-dine-with-me,Lifestyle
31,61824_13,2274,come-dine-with-me,Lifestyle
36,63808_3,5565,come-dine-with-me,Lifestyle
44,51382_23,6192,come-dine-with-me,Lifestyle
45,51383_112,3769,come-dine-with-me,Lifestyle
46,51383_139,786,come-dine-with-me,Lifestyle
47,51383_16,7133,come-dine-with-me,Lifestyle


In [103]:
brand_movie_genre[brand_movie_genre["conEpIndex"]==2495]

,conEpID,conEpIndex,brand,genre
4,47822_100,2495,come-dine-with-me,Lifestyle


In [7]:
num_videos = len(y_tr_p_w.iloc[1,3:-1])

In [8]:
utility_matrix = (y_tr_p_w.groupby('user_id').sum()).reset_index()


In [9]:
utility_matrix = utility_matrix[(utility_matrix["user_id"].isin(user_tr_choice))| 
                                (utility_matrix["user_id"].isin(user_te_choice))]

In [16]:
utility_matrix[:3]

,movie_id,rank,vid__0.0,vid__1.0,vid__2.0,vid__3.0,vid__4.0,vid__5.0,vid__6.0,vid__7.0,...,vid__7729.0,vid__7730.0,vid__7731.0,vid__7732.0,vid__7733.0,vid__7734.0,vid__7735.0,vid__7736.0,vid__7737.0,desired
user_id,,,,,,,,,,,,,,,,,,,,,
0.0,969678.0,28435.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,967779
1.0,936831.0,34710.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,934964
2.0,811861.0,24525.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,810324
3.0,1160869.0,52644.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1158619


In [10]:
utility_matrix_tr  = tr_y.groupby('user_id').sum()

In [11]:
utility_matrix_te = te_y.groupby('user_id').sum()

In [12]:
_utility_matrix = utility_matrix.reset_index().iloc[:,3:-1]

In [13]:
_utility_matrix_tr = utility_matrix_tr.reset_index().iloc[:,3:-1]

In [14]:
_utility_matrix_te = utility_matrix_te.reset_index().iloc[:,3:-1]

In [18]:
n_users = num_users
top_k = 15
top_r = 5
top_l = 10

In [36]:
tf.reset_default_graph()

In [37]:
item_utility_mat = tf.placeholder(tf.float32, shape=[tr_users ,num_videos], name="input_placeholder_a")
top_all_in = tf.placeholder(tf.int32, shape=[num_videos, num_videos], name='sim_mat')
top_k_in = tf.placeholder(tf.int32, shape=[num_videos ,top_k], name="similarity_matrix_k")
top_r_in = tf.placeholder(tf.int32, shape=[num_videos ,top_r], name="similarity_matrix_r")
top_l_in = tf.placeholder(tf.int32, shape=[num_videos ,top_l], name="similarity_matrix_l")

user_desire = tf.placeholder(tf.int32, shape=[None ,1], name="user_des")

In [38]:
normalize_b = tf.nn.l2_normalize(item_utility_mat, 0)
normalize_a = tf.transpose(normalize_b)        
cos_similarity =(tf.matmul(normalize_a, normalize_b))
top_vals_all ,top_all_indice = tf.nn.top_k(cos_similarity, num_videos, sorted=True)
top_vals_k ,top_k_indice = tf.nn.top_k(cos_similarity, top_k, sorted=True)
top_vals_r ,top_r_indice = tf.nn.top_k(cos_similarity, top_r, sorted=True)
top_vals_l ,top_l_indice = tf.nn.top_k(cos_similarity, top_l, sorted=True)

In [39]:
with tf.name_scope('user_accuracy'):
    user_pred_r = tf.reshape(tf.gather(params = top_r_in, indices = user_desire), [-1, top_r])
    to_cast_r = tf.cast(tf.equal(user_desire[1:], user_pred_r[:-1]), tf.float32)
    user_accuracy_r = tf.reduce_sum(tf.reduce_sum(to_cast_r, 1))
    
    user_pred_l = tf.reshape(tf.gather(params = top_l_in, indices = user_desire), [-1, top_l])
    to_cast_l = tf.cast(tf.equal(user_desire[1:], user_pred_l[:-1]), tf.float32)
    user_accuracy_l = tf.reduce_sum(tf.reduce_sum(to_cast_l, 1))

    user_pred_k = tf.reshape(tf.gather(params = top_k_in, indices = user_desire), [-1, top_k])
    to_cast_k = tf.cast(tf.equal(user_desire[1:], user_pred_k[:-1]), tf.float32)
    user_accuracy_k = tf.reduce_sum(tf.reduce_sum(to_cast_k, 1))

with tf.name_scope('mean_reciprocal_rank_with_top_k'):
    user_pred_all = tf.reshape(tf.gather(params = top_all_in, indices= user_desire), [-1,num_videos])
    to_cast = tf.cast(tf.equal(user_desire[1:], user_pred_all[:-1]), tf.float32)
    to_where = (tf.where(tf.equal(to_cast, tf.ones(shape=[tf.shape(to_cast)[0], 1]))))[:,1]

In [25]:
np.diag(cos_sim)

array([ 1.00000024,  0.99999976,  0.99999964, ...,  0.99999982,
        1.00000012,  0.99999976], dtype=float32)

In [45]:
cos_sim

array([[ 1.00000024,  0.03187074,  0.06234039, ...,  0.02825039,
         0.04507203,  0.0495001 ],
       [ 0.03187074,  0.99999976,  0.04445541, ...,  0.04029115,
         0.03214122,  0.        ],
       [ 0.06234039,  0.04445541,  0.99999964, ...,  0.03940551,
         0.        ,  0.0276184 ],
       ..., 
       [ 0.02825039,  0.04029115,  0.03940551, ...,  0.99999982,
         0.05698029,  0.        ],
       [ 0.04507203,  0.03214122,  0.        , ...,  0.05698029,
         1.00000012,  0.05990423],
       [ 0.0495001 ,  0.        ,  0.0276184 , ...,  0.        ,
         0.05990423,  0.99999976]], dtype=float32)

In [72]:
brand_movie_genre["conEpIndex"] = brand_movie_genre["conEpIndex"].astype(int)

In [58]:
black_mirror = list(t_k_idx[512,:])

In [80]:
count_down = t_k_idx[5006,:]

In [104]:
come_dine_with_me = t_k_idx[2495,:]

In [113]:
my_mad_fat_diary = t_k_idx[6669,:]

In [70]:
black_mirror = np.array(black_mirror)

In [109]:
black_mirror_trial = (brand_movie_genre[brand_movie_genre["conEpIndex"].isin(black_mirror)])

In [105]:
come_dine_trial = (brand_movie_genre[brand_movie_genre["conEpIndex"].isin(come_dine_with_me)])

In [81]:
count_down_trial = (brand_movie_genre[brand_movie_genre["conEpIndex"].isin(count_down)])

In [114]:
my_mad_fat_trial = (brand_movie_genre[brand_movie_genre["conEpIndex"].isin(my_mad_fat_diary)])

In [115]:
sort_pd_df_by_ext_vec(my_mad_fat_trial, my_mad_fat_diary, ["conEpIndex"])#[["brand","genre"]]

,conEpID,conEpIndex,brand,genre
0,50688_3,6669,my-mad-fat-diary,Drama
1,63432_19,6635,four-in-a-bed,Lifestyle
2,49114_2,515,black-mirror,Drama
3,49114_3,512,black-mirror,Drama
4,26408_3,518,the-adam-and-joe-show,Comedy
5,61486_48,639,kotv-boxing-weekly,Sports
6,44894_21,958,come-dine-with-me,Lifestyle
7,62623_9,1123,10,Drama
8,58580_3,2108,married-at-first-sight,Factual
9,61824_18,2529,come-dine-with-me,Lifestyle


In [65]:
(brand_movie_genre[brand_movie_genre["conEpIndex"].isin(black_mirror)])#[["brand","genre"]]

,conEpID,conEpIndex,brand,genre
483,62521_2,3062.0,stand-up-to-cancer,Entertainment
1347,61593_3,2006.0,drifters,Comedy
2132,56772_1,1115.0,double-your-house-for-half-the-money,Lifestyle
3660,63432_19,6635.0,four-in-a-bed,Lifestyle
3999,51383_71,2750.0,come-dine-with-me,Lifestyle
4080,53564_16,4708.0,unreported-world,Factual
4215,64970_1,2553.0,no-limit,Drama
4743,61830_13,2600.0,food-unwrapped,Factual
5294,49114_3,512.0,black-mirror,Drama
5491,50688_3,6669.0,my-mad-fat-diary,Drama


In [40]:
sess = tf.Session()
t_r_idx, t_k_idx, t_l_idx, t_all_idx ,cos_sim = sess.run([top_r_indice 
                                      ,top_k_indice, top_l_indice , top_all_indice, cos_similarity], 
                                     feed_dict={item_utility_mat: _utility_matrix_tr})

In [41]:
with open('col_filter_item.csv', 'wb') as csvfile:
    wr = csv.writer(csvfile, delimiter='\t', lineterminator='\n')
    use_accu_r = []
    use_accu_l = []
    use_accu_k = []
    use_mrr = []
    for user in (te_y["user_id"].unique()):
        _user = (te_y[te_y['user_id']==user]['desired']).reshape(-1, 1)
        pred_k, tr_mrr, acc_r, acc_k, acc_l  = sess.run([user_pred_k,to_where, user_accuracy_r, user_accuracy_k, user_accuracy_l]
                                                , feed_dict = {top_r_in: t_r_idx
                                                      , top_k_in: t_k_idx, top_l_in: t_l_idx, 
                                                                                 top_all_in: t_all_idx,  
                                                                                  user_desire: _user})
        
        use_mrr.append(tr_mrr)
        use_accu_r.append(acc_r)
        use_accu_l.append(acc_l)
        use_accu_k.append(acc_k)
        wr.writerow([acc_r])
        wr.writerow([acc_k])
        wr.writerow([tr_mrr])
    print ('Accuracy k:', sum(use_accu_k) / (te_y['desired'].shape[0]) )
    print ('Accuracy l:', sum(use_accu_l) / (te_y['desired'].shape[0]) )
    print ('Accuracy r:', sum(use_accu_r) / (te_y['desired'].shape[0]) )
    print ('MRR:', use_mrr )

/usr/lib/python2.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


('Accuracy k:', 0.5112760684824551)
('Accuracy l:', 0.48397134871510528)
('Accuracy r:', 0.41590757358016589)
('MRR:', [array([   4,    6,    7,  950, 1338, 1157, 1747, 5431, 3938,    2,    2,
          1,    3,   60,    1,    2,    1,  333, 2798, 6280,  200,    1,
        258,    1,    1,    1, 1147,    2,    2,    4,    3,    1,    2,
          3,   11, 1718, 2915,  157, 2466,  884,    1,    2,    2, 2031,
         11,   19,    4,   48, 7289, 2697, 2371,    1,    1,    2,    6,
          3,    1,    3,    5,    3,    3,    1,    2,    2,   10,    1,
          2,    9,   15,    1,    2,    2,    1,    6,    1,    1,    3,
          2,    1,    2,    4,    2,    1,    3,  199,    1,    2,    2,
         54,    1,    3,    1,   19,   78,    1,    1,    2,    3,   10,
          1,    1,    2,   21,    2,    4, 5480, 4462,    2,    1,    4,
         16,    1,    2,    1,    3,    7,    2,    4,   21,    2,    1,
          2, 2164, 3152,    7,    1,    2,    3,    5,    1,    1,   20,
    

In [118]:
use_mrr[1]

array([   1,    1,    1,   67, 1750, 6043,  123, 1098,    2,    2,    2,
         15,    1,    3,    2,  127,  477,    1,    2, 1337,    1,  764,
          2,    7,    4,    6,    5,    3,    3,  536,    1,   22,    1,
          2,    1,    2,  447,    1,    3,    3,  238,    4,    1,    5,
          1,    1,    2,    3,  148,    1,    5,  235,    2,    1,  637,
          1,    3,    1, 2331, 3744, 3631, 1454,    2,    4,    2,    2,
          5,    1,    4,    1,    1,    2,  496,    1,    3,    1,    2,
          1,    1,    1,    2,    4,    1,    1,    2,    4,    1,    5,
          1,    3,   10,    2,    4,    3,   15,    1,    2,    3,    2,
        172,    1,    1,    1,  433,    1,    2,    1,   29,  474,    3,
       1207,    1,    2,    2,   31,   11, 1841,    1,    2,    2,    1,
          7,    1,    2,    2,  114,  496,    2,    3, 6878,   13,    3,
         50,    3,   13, 3065, 2583, 2128,    3,    9,   23, 5345, 5411,
       5400, 5394, 5393, 2515,    1, 6856, 2827,  2

In [120]:
len(use_mrr)

364

In [141]:
mean_mrr = []
for user in range(len(use_mrr)):
    mean_mrr.append(np.percentile(use_mrr[user],50))

In [153]:
np.mean(mean_mrr)

106.27060439560439

In [149]:
use_mrr

[array([   4,    6,    7,  950, 1338, 1157, 1747, 5431, 3938,    2,    2,
           1,    3,   60,    1,    2,    1,  333, 2798, 6280,  200,    1,
         258,    1,    1,    1, 1147,    2,    2,    4,    3,    1,    2,
           3,   11, 1718, 2915,  157, 2466,  884,    1,    2,    2, 2031,
          11,   19,    4,   48, 7289, 2697, 2371,    1,    1,    2,    6,
           3,    1,    3,    5,    3,    3,    1,    2,    2,   10,    1,
           2,    9,   15,    1,    2,    2,    1,    6,    1,    1,    3,
           2,    1,    2,    4,    2,    1,    3,  199,    1,    2,    2,
          54,    1,    3,    1,   19,   78,    1,    1,    2,    3,   10,
           1,    1,    2,   21,    2,    4, 5480, 4462,    2,    1,    4,
          16,    1,    2,    1,    3,    7,    2,    4,   21,    2,    1,
           2, 2164, 3152,    7,    1,    2,    3,    5,    1,    1,   20,
        1263,  510,  139, 3857,   27,    1,    1,    2,    8,    3,    3,
        1529, 3715,    3,    7,    1, 